# 컴퓨터 비전 분야 백본 네트워크
AlexNet, VGG, ResNet 등 딥러닝 네트워크 구조를 알아볼 것


---

### 학습 목표
* 딥러닝 논문의 구조
* ResNet의 핵심 개념과 그 효과
* ResNet 이후 시도 (1) Connection을 촘촘히
* ResNet 이후 시도 (2) 어떤 특성이 중요할까?
* 모델 최적화하기 (1) Neural Architecture Search
* 모델 최적화하기 (2) EfficientNet
* 직접 찾아보기
---

## 1. ResNet 살펴보기
2015년 발표된 ResNet의 원본 논문은 Deep Residual Learning for Image Recognition 이라는 제목으로 Kaiming He, Xiangyu Zhang 등이 작성<br>
Residual Block이라는 아주 간단하면서도 획기적인 개념을 도입하여, Model의 Layer가 깊어져도 안정적으로 학습하면서 성능 개선까지 입증함.

> [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385)

### 1-1. 기존 문제
* 레이어를 깊이 쌓았을 때 Vanishing/Exploding Gradient 문제가 발생하여 모델의 수렴을 방해하는 문제 => 이는 원래 몇 가지 대응 방법이 알려져 있음.
> normalized initialization, intermediate normalization layers 등<br>
* <핵심 문제는 바로> **Degradation Problem** : Vanishing/Exploding Gradient 문제와는 달리, 레이어를 깊이 쌓았을 때 모델이 수렴하고 있음에도 불구하고 발생하는 문제임.
* 이는 Layer depth가 깊어졌을 때 모델이 수렴했음에도 불구하고, 오히려 Layer 개수가 적을 때보다 모델의 training/test error가 커지는 현상을 의미함.(weight 분포가 균등하지 않고, 역전파 시 기울기도 충분하지 않아 학습이 안정적으로 진행되지 못함)




![ResNet_1](ResNet_1.png)

### 1-2. ResNet이 제시한 해결책
#### => 레이어의 입력값을 활용하여 레이어의 "residual function"(잔차 함수)을 학습시킨다.
* 지름길("shortcut connection")을 통해서 레이어가 입력값을 직접 참조하도록 레이어를 변경한 구조를 가짐
![Residual learning](ResNet_2.png)

* 간단히 말하면 학습할 레이어 <span style="color:red">**H(x)**를 **F(x)+x**</span>로 만든 것
* 설령 F(x)가 Vanishing Gradient 현상으로 전혀 학습이 안 되어 zero mapping이 될지라도, H(x)는 최소한 identity mapping이 되므로 최종적으로 성능 저하는 일어나지 않는 것.
* 이후 ReLU 활성함수(activation function)를 거치게 됨.<br>

* $F(x,W_{i})$는 학습시킬 residual mapping으로 잔차 학습이 이 식을 학습함.
---

### 1-3. ResNet 실험 결과
* 우선 Block이란 개념을 알아야 한다.
> Deep Learning의 Deep Network를 쌓는 과정에서 특정 레이어 패턴이 반복되는 경우가 생김. 이 패턴을 묶고, 각 구조에서 조금씩 바뀌는 부분을 변수로 조정할 수 있게끔 만든 모듈이 바로 **"Block"**
* shortcut connection의 효과를 입증하려면, shortcut connection이 있는 네트워크와 없는 네트워크 / 네트워크가 얕은, 깊은 경우로 성능을 비교하면 된다.
* **실험에서는 <span style="color:green">18개 층과 34개 층을 갖는 네트워크</span>, <span style="color:blue">shortcut이 없는(plain network), 있는(Residual network) 네트워크</span> => 총 4가지를 만들어 성능을 비교하였음**
![ResNet Layer 18,34](ResNet_3.png)
<center><span style="color:silver">얇은 curve는 error를 나타내고, 굵은 curve는 validation error를 나타냄.<span style="color:green"></center>
<center>(왼쪽: plain network of 18 and 34 layers. 오른쪽: ResNet of 18 and 34 layers)</center>


> 왼쪽: **plain network**는 layer가 많아져도(깊어져도) 오히려 error rate가 높아진 것이 확인됨.<br>
> 오른쪽: 하지만 shortcut이 적용된 **ResNet**은 layer가 많아지자 더 학습이 잘 되었음.


![Plain_ResNet_Top-1_error](ResNet_4.png)
<center>(Top-1 error on ImageNet validation)</center>


> Top-1 error란 모델이 가장 높은 확률 값으로 **예측한 class 1개가 정답과 일치하는지** 보는 경우의 error rate (당연히 낮을수록 좋음)<br>
(Top-5: 모델이 예측한 값들 중 가장 높은 확률 값부터 순서대로 5개 class 중 정답이 있는지를 보는 것)<br>
> 마찬가지로 왼쪽(plain network)는 layer가 깊어지자 오히려 error rate가 증가, 반대로 **ResNet은 layer가 깊어지면 error rate도 감소**함.

---
## 2. DenseNet 살펴보기
2016년 발표된 **Densely Connected Convolutional Networks**로 <U>ResNet의 **shortcut connection**을 마치 **Fully Connected Layer처럼 촘촘히 가지도록**</U> 한다면 더욱 성능 개선 효과가 클 것이라고 생각.
> [Densely Connected Convolutional Networks (원 논문)](https://arxiv.org/abs/1608.06993)<br>
> [DenseNet Tutorial 1 Paper Review & Implementation details (한글 리뷰)](https://hoya012.github.io/blog/DenseNet-Tutorial-1/)

## 2-1. 기본 컨셉
![DenseNet](DenseNet_1.png)
<center>(growth rate of k =4인 5개의 dense block)</center>

* 일반적인 CNN이 $L$개의 Layer에 대해 각 Layer당 한 개씩 연결, 즉 $L$개의 연결을 갖는 것과 달리, DenseNet의 기본 Block은 $L$개의 Layer가 있다면 Layer간 ${{L(L+1)} \over {2}}$의 직접적인 연결(direct connection)을 생성함.<br>
* 이러한 연결 구조를 **"dense connectivity"**라고 부르며, $H_{1}$로 표기.

#### Dense connetivity
><center>$X_{l} = {H}_{l}([X_{0},X_{1},...,X_{l-1}])$</center>
>* 합성함수(composite function)로, 이전 Layer에서 나온 특성 맵(feature map) 전부를 input으로 받음.<br>
>* $X_{0}$(0번째 layer를 거친 feature map)부터 $X_{l-1}$($l-1$번째 layer를 거친 feature map)이 모두 합성함수를 거쳐 $l$번째 layer의 출력값이 됨.<br>
>* 이를 통해 경사 소실 문제(gradient vanishing)를 개선하고 특성을 계속 재사용할 수 있음

## 2-2 ResNet과의 차이
* ResNet은 shortcut을 원소별로 **단순히 더해주었던** 반면, DenseNet은 하나하나 **특성 맵을 채널 방향으로 쌓아서(concatenate)** 하나의 텐서로 만들어 낸다는 점이 다름.
* 또한 ResNet은 connection에서 다른 연산이 없었던 것과 달리, 합성함수 $H_{l}$ 은 이 텐서에 대해 배치 정규화(batch normalization, BN), ReLU 활성화 함수, 그리고 3x3 컨볼루션 레이어를 통해서 pre-activation을 수행함.<br><br>


* Pre-Activation ResNet
![Pre-activation](DenseNet_2.png)
<center><span style="color:silver">[Identity Mappings in Deep Residual Networks 논문]</span><center>
<center>(우측: Pre-activation / 점선은 training loss, 굵은 선은 test error)</center>


> Pre-activation이란 ReLU가 컨볼루션 블록 안으로 들어간 것을 의미. 즉 기존의 residual block 구조에 활성화 함수의 순서를 바꾼 것으로 성능을 올린 것.<br>
> 입력값을 BN(Batch Normalization)과 ReLU 활성화 함수를 거친 뒤에 convolution layer에 전달함.<br>
> [pre-activation RestNet 리뷰](https://deep-learning-study.tistory.com/510)

---
### 2-3. growth rate
#### 예제(growth rate가 필요한 이유)
특성 맵을 더해주던 ResNet과 달리 DenseNet에서는 특성 맵을 채널 방향으로 쌓아서 사용한다.<br>
그렇다면 **4개의 Channel을 가진 CNN 레이어 4개**를 DenseNet block으로 만들었을 때, <U>input의 채널 개수가 4인 경우</U> block 내 각 Layer의 input은 몇 개의 Channel을 가지는가?<br>

* 첫 번째 layer input의 Channel은 입력 데이터의 Channel 그대로 4.
* 두 번째 layer input은 input data의 channel 값 4와, 첫 번째 layer의 출력값의 channel 값인 4를 더해 8.
* 세 번째 layer input은 input data의 channel 값 4와, 첫 번째, 두 번째 layer의 출력값의 channel 4+4로 총 8을 받아 12.
* 네 번째는 같은 방식으로 16.


이처럼 input의 channel은 4로 시작했지만, feature map의 크기가 매우 커지기 때문에, **growth rate**라는 값을 조정하여 channel의 개수를 조절함.<br><br>


#### Imagenet 챌린지에서 DenseNet의 growth rate를 32로 사용했음.<br> 아래 예시는 12개의 convolution layer가 있는 두 번째 dense block을 구현할 때 각 레이어가 입력받는 channel이 몇 개인지 확인하는 코드.

In [1]:
# Dense Block내의 각 레이어 output의 channel을 계산하는 함수
def get_channel_list():
    channel_list = []
    input_channel = 32
    growth_rate = 32
    for i in range(12):
        channel_list.append(input_channel + growth_rate*i)
    return channel_list

get_channel_list()

[32, 64, 96, 128, 160, 192, 224, 256, 288, 320, 352, 384]

---
## 3. SeNet 살펴보기
2017년 나온 네트워크로 Squeeze-and-Excitation Networks의 줄임말. **Squeeze**란 말 그대로 feature에서 중요한 정보를 짜내게 됨.<br>
* 일반적인 CNN은 컨볼루션 필터를 filter size에 따라 적용하므로, 필터의 개수가 곧 convolution layer 출력의 channel 개수가 됨.<br>
* 하지만 SqueezeNet에서는 이때 채널 방향으로 global average pooling을 적용하고, 압축된 정보를 활용하여 중요한 채널이 활성화되도록 만듦.


![Global Average Pooling](GAP.png)
<center>[Global Average Pooling 과정]</center>

#### Squeeze
그렇다면 feature map의 channel에서 어느 채널이 중요한지 알기 위해서는 어떻게 해야 할까?<br>
=> 우선 채널 정보를 압축해서 가져오고 분석하자. 따라서 여기서 **Pooling**이 사용된다.
> 복습: Pooling은 CNN에서 주로 커널(kernel) 영역의 정보를 압축하는 데 사용했음. 커널 영역의 최댓값만 남기는 것이 Max Pooling, 평균값을 남기는 것이 Average Pooling.


![SeNet block](SeNet_1.png)
<center>[A Squeeze-and-Excitation block]</center>
* $F_{sq}$ 함수에서 Squeeze가 일어남.<br>
* $F_{tr}$이라는 convolution layer를 거치면 "HxWxC" size의 Feature map U가 나오고, U에 Squeeze를 적용하면 "1x1xC" size로 나오게 된다.<br>
* 각 채널별로 딱 1개의 숫자만 남도록 2D 특성맵 전체에 대해 평균값을 남기는 global average pooling을 적용.

#### Excitate
channel 정보를 압축해서 가져왔으니, 이제 어떤 channel이 중요할지 판단해야 함.(이것이 **excitation**)
>$$ s= F_{ex}(z, W)= \sigma(g(z, W)) = \sigma(W_{2}\delta(W_{1}z)) $$<br>
> (z: global average pooling를 적용한 feature, 즉 squeeze한 결과물)<br>
> 이 feature에 **$W_{1}$**을 곱해주는 linear layer를 거치고, **ReLU 활성화 함수** $\delta$를 거침.<br>
> 이후 **$W_{2}$**을 곱해주는 linear layer를 거치고, 마지막으로 **sigmoid 활성화 함수** $\sigma$를 거침.<br>


>이때 sigmoid를 사용하는 이유는 가장 중요한 하나의 채널만 활성화되는 것이 아닌, **여러 채널**들이 서로 다른 정도로 활성화되도록 하기 위함.<br>
>> 데이터셋에 <U>정답 label이 하나</U>뿐인 단순 분류 모델의 활성화 함수로는 SoftMax를 사용해서 단 하나의 최댓값을 찾지만,<br>
>> 하나의 대상에도 <U>여러 개의 클래스의 정답 label</U>을 지정할 수 있는 다중 라벨 분류(multi label classification)에서는 sigmoid를 사용하는 것과 같은 맥락

---
## 4. NASNet 살펴보기
NAS(neural architecture search) : 신경망을 사용해 모델의 구조를 탐색하는 접근 방식.<br>
2017년 등장. 딥러닝 모델의 파라미터(parameter)를 최적화해 왔듯이 model의 구조 자체도 최적화할 수는 없을지 여부를, 신경망을 사용해 탐색.
> [Learning Transferable Architectures for Scalable Image Recognition (원 논문)](https://arxiv.org/abs/1707.07012)<br>
> [Neural Architecture Search With Reinforcement Learning (한글 리뷰)](http://solarisailab.com/archives/2691)<br>


학습 모델이 대상 신경망의 구성을 일종의 변수로 조정하면서 최적의 성능을 내도록 함. 지금까지 보아왔던 레이어의 세부 구성, CNN의 filter size, channel의 개수, connection 등을 조정할 수 있는 변수로 둠.<br>
> 이렇게 네트워크 구성에 대한 요소들을 조합할 수 있는 범위를 탐색 공간(search space)이라고 함.


 * 물론 layer 하나하나의 하이퍼 패러미터를 조절하려면 탐색 공간이 너무 방대해지게 됨.<br>
 * 따라서 NASNet에서는 탐색 공간을 줄이기 위해 **모듈(cell) 단위의 최적화**를 진행하고, 그 **모듈을 조합**하는 방식으로 진행함.
 * ResNet에서는 'Residual Block', DenseNet에서는 'Dense Block' 모듈이었다면, NASNet에서는 convolution cell이라고 부름.

#### convolution cell
normal cell과 reduction cell로 구분됨.
* Normal cell은 특성 맵의 가로, 세로가 유지되도록 stride를 1로 고정함.<br>
* Reduction cell은 stride를 1 또는 2로 가져가서 특성 맵의 크기가 줄어들게 함.
* normal cell과 reduction cell 내부만을 최적화해 두 가지 cell을 조합하여 최종 결과 네트워크(NASNet)이 나오게 된 것.


![NASNet](NASNet.png)

---
## EfficientNet 살펴보기
2019년 발표. CNN에서 **width, depth, resolution** 세 가지 요소에 집중하여 최적화한 모델.
> [EfficientNet: Rethinking Model Scaling for Convolutional Neural Networks (원 논문)](https://arxiv.org/abs/1905.11946)<br>
> [EfficientNet 한글 리뷰](https://hoya012.github.io/blog/EfficientNet-review/)<br>

* width: CNN channel의 size<br>
* depth: network의 depth<br>
* resolution: input의 width(w)와 height(h)

![EfficientNet Accuracy](EfficientNet_1.png)


#### Compound scaling
우선 resolution, depth, width 조절 전, NASNet과 유사한 방법을 사용해 기본 모델(baseline network)의 구조를 미리 찾아서 고정함.<br>
이후 개별 레이어의 resolution, depth, width 를 조절해 기본 모델을 적절히 확장시키는 문제를 해결.<br>
그래서 resolution, depth, width라는 세 가지 "scaling factor"는 어떻게 찾는가? => **compound scaling**을 이용<br><br>


![compound scaling](EfficientNet_2.png)
<center>(compound scaling)</center>


* compound coefficient $\phi$: 모델의 크기를 조정하기 위한 계수.<br>
(고정된 계수 $\phi$에 따라 resolution, depth, width를 조절해서, 일정한 규칙에 따라 모델의 구조를 조절할 수 있음.)<br>
* $\phi$를 1로 고정한 뒤 resolution, depth, width를 정하는 $\alpha, \beta, \gamma$ 최적값을 찾음 (그리드 탐색)<br>
* 이후 $\alpha, \beta, \gamma$의 배율을 고정한 뒤, compound coefficient $\phi$를 조절하여 모델의 크기를 조정.


---